# Decrypting Signal Backup

In [ ]:
from cryptography.hazmat.primitives          import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends            import default_backend
from cryptography.hazmat.primitives.ciphers  import Cipher, algorithms, modes

import Backups_pb2
BACKUP_PASSPHRASE = '770374738950407044994423070922'
BACKUP_FILE       = 'signal.backup'

def getBackupKey(passphrase, salt):

    i = bytes(passphrase, encoding='utf-8')
    h = i

    digest = hashes.Hash(hashes.SHA512(), backend=default_backend())
    digest.update(salt)

    for k in range(250000):
        digest.update(h + i)
        h = digest.finalize()
        digest = hashes.Hash(hashes.SHA512(), backend=default_backend())

    return h[:32]

## Load encrypted frames

In [ ]:
f = open(BACKUP_FILE, "rb")
    
header_length_bytes = f.read(4)
header_length       = int.from_bytes(header_length_bytes, byteorder='big')
header_frame        = f.read(header_length)

#### Get AES init vector and passphrase salt

In [ ]:
frame = Backups_pb2.BackupFrame()
frame.ParseFromString(header_frame)

iv   = frame.header.iv
salt = frame.header.salt

## Decrypt frames

In [ ]:
key = getBackupKey(BACKUP_PASSPHRASE, salt)

derived = HKDF(
    algorithm=hashes.SHA256(),
    length=64,
    salt=None,
    info=bytes('Backup Export', encoding='utf-8'),
    backend=default_backend()
).derive(key)

cipherKey, macKey = derived[0:32], derived[32:64]

In [ ]:
counter = int.from_bytes(iv[:4], byteorder='big')

frames = []

while True:
    
    chunk = f.read(4)
    
    if chunk:
    
        length    = int.from_bytes(chunk, byteorder='big')
        frame     = f.read(length - 10)
        frame_mac = f.read(10)

        # Verify MAC

        mac = hmac.HMAC(macKey, hashes.SHA256(), backend=default_backend())
        mac.update(frame)
        check_mac = mac.finalize()


        if frame_mac != check_mac[:10]:
            raise ValueError('Invalid MAC @ frame ' + str(len(frames)))

        # Decrypt

        iv = counter.to_bytes(length=4, byteorder='big') + iv[4:]
        counter += 1
        cipher = Cipher(algorithms.AES(cipherKey), modes.CTR(iv), backend=default_backend()).decryptor()
        plaintext = cipher.update(frame) + cipher.finalize()

        # Setup protobuf

        frame_pb2 = Backups_pb2.BackupFrame()
        frame_pb2.ParseFromString(plaintext)

        frames.append(frame_pb2)
        
        # In case it's an attachment or an Avatar, it also needs to be decrypted

        if frame_pb2.HasField('attachment') or frame_pb2.HasField('avatar'):

            length = frame_pb2.attachment.length if frame_pb2.HasField('attachment') else frame_pb2.avatar.length

            iv = counter.to_bytes(length=4, byteorder='big') + iv[4:]
            counter += 1
            cipher = Cipher(algorithms.AES(cipherKey), modes.CTR(iv), backend=default_backend()).decryptor()
            mac = hmac.HMAC(macKey, hashes.SHA256(), backend=default_backend())
            mac.update(iv)
            
            enc_file = f.read(length)
            
            mac.update(enc_file)
            check_mac = mac.finalize()
            
            file_mac = f.read(10)
            
            if file_mac != check_mac[:10]:
                raise ValueError('Invalid MAC @ attachment/avatar ' + str(len(frames)))
            
            file = cipher.update(enc_file) + cipher.finalize()
            
            if frame_pb2.HasField('attachment'):
                with open('attachments/' + str(frame_pb2.attachment.rowId) + '_' + str(frame_pb2.attachment.attachmentId), 'wb') as f2:
                    f2.write(file)
            else:
                with open('avatars/' + str(frame_pb2.avatar.name), 'wb') as f2:
                    f2.write(file)
    
    else:
        
        f.close()
        break
        
        

In [ ]:
f = open('statements.sql', 'w')

for frame in frames:
    if frame.HasField('statement'):
        f.write(frame.statement.statement + "\n\n\n")
        
f.close()

In [ ]:
frames[2200].statement